# Agentic Health Triage for Rural Areas

This notebook demonstrates a minimal agentic workflow for basic health
triage designed for low-connectivity environments.


## Objective
Provide basic health orientation and urgency alerts using an offline-capable
agentic workflow built with open models.


# Core imports
import json
from typing import Dict


In [1]:
from dataclasses import dataclass
from typing import List, Optional

@dataclass
class PatientInput:
    age: int
    symptoms: List[str]
    duration_days: int
    chronic_conditions: Optional[List[str]] = None
    pregnant: Optional[bool] = False
    temperature_c: Optional[float] = None


In [2]:
sample_patient = PatientInput(
    age=67,
    symptoms=["febre", "tosse", "falta de ar"],
    duration_days=3,
    chronic_conditions=["hipertensao"],
    temperature_c=38.5
)

sample_patient

PatientInput(age=67, symptoms=['febre', 'tosse', 'falta de ar'], duration_days=3, chronic_conditions=['hipertensao'], pregnant=False, temperature_c=38.5)

In [3]:
#bloco 2 da fase 3 - criação do sistema de score


def calculate_risk_score(patient: PatientInput) -> int:
    score = 0
    
    # Idade
    if patient.age >= 65:
        score += 2
    
    # Febre
    if patient.temperature_c:
        if patient.temperature_c >= 39:
            score += 3
        elif patient.temperature_c >= 38:
            score += 2
    
    # Sintomas críticos
    if "falta de ar" in patient.symptoms:
        score += 3
    
    # Doenças crônicas
    if patient.chronic_conditions:
        score += 2
    
    # Duração prolongada
    if patient.duration_days > 7:
        score += 1
    
    return score



In [4]:
def classify_risk(score: int) -> str:
    
    if score >= 7:
        return "CRITICAL"
    
    if score >= 5:
        return "HIGH"
    
    if score >= 3:
        return "MEDIUM"
    
    return "LOW"


In [5]:
score = calculate_risk_score(sample_patient)
risk_level = classify_risk(score)

print("Risk Score:", score)
print("Risk Level:", risk_level)


Risk Score: 9
Risk Level: CRITICAL


In [6]:
#bloco 03

def generate_risk_explanation(patient: PatientInput, score: int) -> str:
    
    reasons = []
    
    if patient.age >= 65:
        reasons.append("Idade avançada (≥65 anos)")
    
    if patient.temperature_c:
        if patient.temperature_c >= 39:
            reasons.append("Febre alta (≥39°C)")
        elif patient.temperature_c >= 38:
            reasons.append("Febre moderada (≥38°C)")
    
    if "falta de ar" in patient.symptoms:
        reasons.append("Presença de falta de ar")
    
    if patient.chronic_conditions:
        reasons.append("Histórico de condições crônicas")
    
    if patient.duration_days > 7:
        reasons.append("Sintomas persistentes por mais de 7 dias")
    
    explanation = "Classificação baseada nos seguintes fatores: " + ", ".join(reasons)
    
    return explanation


In [7]:
explanation = generate_risk_explanation(sample_patient, score)
print(explanation)


Classificação baseada nos seguintes fatores: Idade avançada (≥65 anos), Febre moderada (≥38°C), Presença de falta de ar, Histórico de condições crônicas


In [8]:
# bloco 04

def generate_recommendation(risk_level: str) -> str:
    
    if risk_level == "CRITICAL":
        return "Encaminhamento imediato para emergência. Procurar atendimento hospitalar urgentemente."
    
    if risk_level == "HIGH":
        return "Recomendado atendimento médico nas próximas horas. Monitorar sinais vitais e sintomas."
    
    if risk_level == "MEDIUM":
        return "Monitoramento domiciliar com orientação médica remota. Retornar se houver piora."
    
    return "Baixo risco. Manter observação e cuidados gerais."


In [9]:
recommendation = generate_recommendation(risk_level)

print("Risk Score:", score)
print("Risk Level:", risk_level)
print("Explanation:", explanation)
print("Recommendation:", recommendation)


Risk Score: 9
Risk Level: CRITICAL
Explanation: Classificação baseada nos seguintes fatores: Idade avançada (≥65 anos), Febre moderada (≥38°C), Presença de falta de ar, Histórico de condições crônicas
Recommendation: Encaminhamento imediato para emergência. Procurar atendimento hospitalar urgentemente.


In [10]:
import pandas as pd
import random

# -----------------------------
# Listas base
# -----------------------------

sintomas = [
    "febre",
    "dor de cabeça",
    "dor no peito",
    "tosse",
    "vomito",
    "falta de ar",
    "diarreia",
    "dor abdominal"
]

intensidades = ["baixa", "media", "alta"]

condicoes = [
    "nenhuma",
    "diabetes",
    "hipertensao",
    "asma",
    "cardiopatia"
]

# -----------------------------
# Definir grupo de risco
# -----------------------------
def definir_grupo_risco(idade, condicao):
    
    if idade >= 60:
        return "sim"
    
    if condicao != "nenhuma":
        return "sim"
    
    return "nao"

# -----------------------------
# Definir nível de risco
# -----------------------------
def definir_nivel_risco(idade, sintoma, intensidade, duracao, grupo_risco):
    
    # Casos críticos imediatos
    if sintoma == "dor no peito" and intensidade == "alta":
        return "alto"
    
    if sintoma == "falta de ar":
        return "alto"
    
    if idade >= 60 and sintoma == "febre" and intensidade == "alta":
        return "alto"
    
    if grupo_risco == "sim" and intensidade == "alta":
        return "alto"
    
    # Casos intermediários
    if duracao >= 5:
        return "medio"
    
    if grupo_risco == "sim" and intensidade == "media":
        return "medio"
    
    if intensidade == "alta":
        return "medio"
    
    # Casos leves
    return "baixo"

# -----------------------------
# Geração dos dados
# -----------------------------

dados = []

for _ in range(80):  # aumentei para mais robustez
    
    idade = random.randint(1, 85)
    sintoma = random.choice(sintomas)
    intensidade = random.choice(intensidades)
    duracao = random.randint(1, 7)
    
    condicao = random.choices(
        condicoes,
        weights=[0.6, 0.1, 0.1, 0.1, 0.1]  # maioria sem condição
    )[0]
    
    grupo_risco = definir_grupo_risco(idade, condicao)
    
    nivel_risco = definir_nivel_risco(
        idade,
        sintoma,
        intensidade,
        duracao,
        grupo_risco
    )
    
    dados.append({
        "IDADE": idade,
        "SINTOMA_PRINCIPAL": sintoma,
        "INTENSIDADE": intensidade,
        "DURACAO_DIAS": duracao,
        "CONDICAO_PREEXISTENTE": condicao,
        "GRUPO_RISCO": grupo_risco,
        "NIVEL_RISCO": nivel_risco
    })

df = pd.DataFrame(dados)

df.head()


,IDADE,SINTOMA_PRINCIPAL,INTENSIDADE,DURACAO_DIAS,CONDICAO_PREEXISTENTE,GRUPO_RISCO,NIVEL_RISCO
0,60,vomito,media,4,diabetes,sim,medio
1,11,dor no peito,media,6,nenhuma,nao,medio
2,55,dor de cabeça,baixa,5,nenhuma,nao,medio
3,45,febre,alta,7,hipertensao,sim,alto
4,47,vomito,alta,7,diabetes,sim,alto


In [11]:
# Core imports
import json
from typing import Dict


In [12]:
def interpret_symptoms(user_input: str) -> Dict:
    return {
        "interpreted_symptoms": user_input.lower(),
        "confidence": "basic"
    }


In [13]:
def classify_risk(symptoms: Dict) -> str:
    text = symptoms["interpreted_symptoms"]
    if "dor no peito" in text or "falta de ar" in text:
        return "high"
    elif "febre" in text:
        return "medium"
    return "low"


In [14]:
def provide_guidance(risk_level: str) -> str:
    if risk_level == "high":
        return "Procure atendimento médico imediato."
    elif risk_level == "medium":
        return "Observe os sintomas e procure um profissional se persistirem."
    return "Repouso e hidratação podem ajudar."


In [15]:
def agentic_triage_pipeline(user_input: str) -> Dict:
    symptoms = interpret_symptoms(user_input)
    risk = classify_risk(symptoms)
    guidance = provide_guidance(risk)
    
    return {
        "risk_level": risk,
        "guidance": guidance
    }


In [16]:
user_input = "Estou com febre e dor no corpo"
result = agentic_triage_pipeline(user_input)
result


{'risk_level': 'medium',
 'guidance': 'Observe os sintomas e procure um profissional se persistirem.'}